# Imports

In [34]:
import requests, json
from pprint import pprint
import pandas as pd
import Player
from importlib import reload
reload(Player)
from Player import Player

In [35]:
pd.set_option('display.max_columns', None)

# URL 

In [36]:
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

## Players

In [37]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# Show id is the PK
print(players['id'].nunique())
print(len(players))

players = players.rename(columns={'id': 'player_id'})
players['player_name'] = players['first_name'] + ' ' + players['second_name']
# show some information about first five players
players[['player_id', 'player_name', 'team', 'element_type']].head()

741
741


,player_id,player_name,team,element_type
0,1,David Raya Martín,1,1
1,2,Kepa Arrizabalaga Revuelta,1,1
2,3,Karl Hein,1,1
3,4,Tommy Setford,1,1
4,5,Gabriel dos Santos Magalhães,1,2


## Teams

In [38]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])
teams = teams.rename(columns={'id': 'team_id'})

teams.sort_values(by='strength')

,code,draw,form,team_id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
2,90,0,None,3,0,Burnley,0,0,16,BUR,2,None,False,0,1050,1050,1050,1050,1050,1050,43
16,56,0,None,17,0,Sunderland,0,0,7,SUN,2,None,False,0,1050,1050,1050,1050,1050,1050,29
10,2,0,None,11,0,Leeds,0,0,12,LEE,2,None,False,0,1050,1075,1050,1050,1050,1100,9
9,54,0,None,10,0,Fulham,0,0,8,FUL,3,None,False,0,1125,1125,1130,1130,1120,1120,34
17,6,0,None,18,0,Spurs,0,0,3,TOT,3,None,False,0,1130,1175,1100,1100,1160,1250,21
15,17,0,None,16,0,Nott'm Forest,0,0,15,NFO,3,None,False,0,1165,1205,1150,1230,1180,1180,15
13,1,0,None,14,0,Man Utd,0,0,11,MUN,3,None,False,0,1105,1125,1110,1110,1100,1140,12
18,21,0,None,19,0,West Ham,0,0,19,WHU,3,None,False,0,1100,1100,1100,1100,1100,1100,25
19,39,0,None,20,0,Wolves,0,0,20,WOL,3,None,False,0,1100,1125,1080,1100,1120,1150,38
7,31,0,None,8,0,Crystal Palace,0,0,5,CRY,3,None,False,0,1140,1160,1120,1130,1160,1190,6


## Positions

In [39]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])
positions = positions.rename(columns={'id': 'position_id'})

positions

,position_id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],86
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],245
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],328
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],82


Merge team, positions and players

In [40]:
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='team_id'
)

df = df.merge(
    positions,
    left_on='element_type',
    right_on='position_id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()
df['now_cost'] = df['now_cost']/10


In [41]:
df['points_per_game'] = df['points_per_game'].astype(float)

In [42]:
import optimisation
import numpy as np
from importlib import reload
reload(optimisation)
from optimisation import TeamOptimizer
sub_df = df[df['minutes']>180]
sub_df = sub_df[sub_df['chance_of_playing_next_round'].isin([np.nan, 100])]

optimizer = TeamOptimizer(sub_df, 'points_per_game')
optimizer.solve()

=== DATA FEASIBILITY CHECK ===
Available Goalkeeper: 19
Available Defender: 74
Available Midfielder: 98
Available Forward: 18
✅ Goalkeeper: 19 >= 2 (OK)
✅ Defender: 74 >= 5 (OK)
✅ Midfielder: 98 >= 5 (OK)
✅ Forward: 18 >= 3 (OK)
Min cost for 2 Goalkeeper players: 8.5
Min cost for 5 Defender players: 20.0
Min cost for 5 Midfielder players: 24.0
Min cost for 3 Forward players: 15.9
Minimum possible team cost: 68.4
Budget: 100
✅ Budget: 68.4 <= 100 (OK)

=== SOLVING OPTIMIZATION ===
Players: 209
Budget: 100
Team size: 15
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/e0065734946f42b7b90971a5371d4ea5-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/e0065734946f42b7b90971a5371d4ea5-pulp.sol (default strateg

{'squad': ['Riccardo Calafiori',
  'Jurriën Timber',
  'Jaidon Anthony',
  'Marcos Senesi Barón',
  'Antoine Semenyo',
  'Bart Verbruggen',
  'Trevoh Chalobah',
  'João Pedro Junqueira de Jesus',
  'Marc Guéhi',
  'Harry Wilson',
  'Ryan Gravenberch',
  'Nico González Iglesias',
  'Nick Pope',
  'Richarlison de Andrade',
  'Niclas Füllkrug'],
 'starters': ['Riccardo Calafiori',
  'Jurriën Timber',
  'Jaidon Anthony',
  'Marcos Senesi Barón',
  'Antoine Semenyo',
  'Trevoh Chalobah',
  'João Pedro Junqueira de Jesus',
  'Marc Guéhi',
  'Ryan Gravenberch',
  'Nick Pope',
  'Richarlison de Andrade'],
 'formation': '5-3-2',
 'captain': 'Antoine Semenyo',
 'vice_captain': 'Jaidon Anthony',
 'total_cost': np.float64(86.2),
 'total_points': np.float64(85.40000000000002)}

# Players Gameweek History

In [43]:
def get_gameweek_history(player_id, players_df = None):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()

    if players_df is not None:
        player = players_df[players_df['player_id']==player_id]['player_name'].values[0]
        print('Gameweek history for ', player)
    
    # extract 'history' data from response into dataframe

    df = pd.json_normalize(r['history'])
    
    return df


# show player #4's gameweek history
gameweek_res = get_gameweek_history(1, players_df=df)
gameweek_res

Gameweek history for  David Raya Martín


,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,1,9,14,10,False,2025-08-17T15:30:00Z,0,1,1,False,90,0,0,1,0,0,0,0,1,0,7,3,38,49.2,0.0,0.0,4.9,1,13,0,0,1,0.00,0.00,0.00,1.52,55,0,1531911,0,0
1,1,11,11,6,True,2025-08-23T16:30:00Z,5,0,2,False,90,0,0,1,0,0,0,0,0,0,1,0,28,13.4,0.0,0.0,1.3,0,3,0,0,1,0.00,0.00,0.00,0.17,55,218659,2284634,277339,58680
2,1,25,12,2,False,2025-08-31T15:30:00Z,1,0,3,False,90,0,0,0,1,0,0,0,0,0,2,0,12,20.0,10.0,0.0,3.0,0,12,0,0,1,0.00,0.02,0.02,0.52,55,-12311,2406964,146739,159050
3,1,31,16,6,True,2025-09-13T11:30:00Z,3,0,4,False,90,0,0,1,0,0,0,0,0,0,1,0,24,12.8,0.0,0.0,1.3,0,9,0,0,1,0.00,0.00,0.00,0.20,55,171289,2765759,289041,117752
4,1,41,13,2,True,2025-09-21T15:30:00Z,1,1,5,False,90,0,0,0,1,0,0,0,0,0,2,0,13,21.4,0.0,0.0,2.1,1,6,0,0,1,0.00,0.01,0.01,0.89,55,-9786,2762632,98100,107886


In [45]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

players = {}
df_list = []

def fetch_player(row):
    player = Player(player_id=row.player_id, merged_df=df)
    player.get_player_gameweek_history()

    player_df = player.gameweek_history.copy()
    player_df["player_id"] = row.player_id
    player_df["player_name"] = row.player_name

    return row.player_name, player, player_df

with ThreadPoolExecutor(max_workers=20) as executor:  # tune worker count
    futures = [executor.submit(fetch_player, row) for row in df.itertuples(index=False)]
    for f in as_completed(futures):
        player_name, player, player_df = f.result()
        players[player_name] = player
        df_list.append(player_df)

player_gw_hist = pd.concat(df_list, ignore_index=True)
player_gw_hist = player_gw_hist.merge(df[['player_id', 'now_cost', 'position_name']],how='left', on='player_id')


Initialised  David Raya Martín
Initialised  Kepa Arrizabalaga Revuelta
Initialised  Karl Hein
Initialised  Tommy Setford
Initialised  Gabriel dos Santos Magalhães
Initialised  William Saliba
Initialised  Riccardo Calafiori
Initialised  Jurriën Timber
Initialised  Jakub Kiwior
Initialised  Myles Lewis-Skelly
Initialised  Brayden Clarke
Initialised  Benjamin White
Initialised  Maldini Kacurri
Initialised  Martin Ødegaard
Initialised  Josh Nichols
Initialised  Noni Madueke
Initialised  Bukayo Saka
Initialised  Gabriel Martinelli Silva
Initialised  Leandro Trossard
Initialised  Declan Rice
Gameweek history for  David Raya Martín
Initialised  Mikel Merino Zazón
Gameweek history for  Martin Ødegaard
Initialised  Fábio Ferreira Vieira
Gameweek history for  Riccardo Calafiori
Initialised  Christian Nørgaard
Gameweek history for  Gabriel Martinelli Silva
Gameweek history for  Josh Nichols
Gameweek history for  Gabriel dos Santos Magalhães
Initialised  Martín Zubimendi Ibáñez
Initialised  Ismeal

In [56]:
def get_most_selected_team_for_week(week, gameweek_history):
    gameweek_history = gameweek_history[gameweek_history['round']==week]

    optimizer = TeamOptimizer(gameweek_history, 'selected')
    res = optimizer.solve()
    
    squad = res['squad']

    total_points = 0
    for player in squad:
        total_points+=gameweek_history[gameweek_history['player_name']==player]['total_points'].values[0]
    print(total_points)
    return res

In [64]:
res = get_most_selected_team_for_week(5, player_gw_hist)

=== DATA FEASIBILITY CHECK ===
Available Goalkeeper: 86
Available Defender: 245
Available Midfielder: 328
Available Forward: 82
✅ Goalkeeper: 86 >= 2 (OK)
✅ Defender: 245 >= 5 (OK)
✅ Midfielder: 328 >= 5 (OK)
✅ Forward: 82 >= 3 (OK)
Min cost for 2 Goalkeeper players: 7.9
Min cost for 5 Defender players: 19.5
Min cost for 5 Midfielder players: 22.0
Min cost for 3 Forward players: 13.1
Minimum possible team cost: 62.5
Budget: 100
✅ Budget: 62.5 <= 100 (OK)

=== SOLVING OPTIMIZATION ===
Players: 741
Budget: 100
Team size: 15
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/3990bfcd0fb544e9ac78490592b31853-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/3990bfcd0fb544e9ac78490592b31853-pulp.sol (default str

Strategies
1. Simple expected points calculation
2. Following the trend of other players
3. Machine learning prediction

In [66]:
my_team_points=[45,
63,
44 ,
44,
39]

most_selected_team_points =[
31,
46,
48,
80,
47]

In [70]:
import plotly.graph_objects as go

points_df = pd.DataFrame({
    "Gameweek": range(1, len(my_team_points)+1),
    "My Team": my_team_points,
    "Most Selected Team": most_selected_team_points
})

# --- Plotly Line Chart ---
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=points_df["Gameweek"],
    y=points_df["My Team"],
    mode="lines+markers",
    name="My Team"
))

fig.add_trace(go.Scatter(
    x=points_df["Gameweek"],
    y=points_df["Most Selected Team"],
    mode="lines+markers",
    name="Most Selected Team"
))

fig.update_layout(
    title="Points per Gameweek",
    xaxis_title="Gameweek",
    yaxis_title="Points",
    template="plotly_white",
    legend=dict(x=0, y=1, bgcolor="rgba(0,0,0,0)")
)